# Ordering a list of clustered days

* recieve a list called Labels containtng 365 entries, 1 for each day in a year
* values in Labels go from cluster 0 to cluster n
* the goal is order the clusters of days according to the list Labels only

In [1]:
# The numpy package is necessary 
import numpy as np

# A list with 17 clusters generated by the k-means algorythme using sklearn
Labels = [7, 7, 2, 2, 2, 7, 7, 7, 7, 2, 2, 7, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 
          2, 2, 7, 7, 7, 2, 2, 2, 7, 7, 7, 2, 2, 2, 2, 7, 13, 13, 7, 7, 7, 12, 
          12, 12, 12, 12, 0, 12, 12, 13, 12, 12, 12, 12, 12, 0, 13, 13, 0, 13, 
          12, 12, 0, 13, 13, 0, 7, 12, 7, 7, 13, 4, 0, 7, 12, 12, 12, 12, 4, 4, 
          0, 12, 12, 12, 4, 4, 4, 4, 4, 0, 7, 12, 12, 0, 0, 0, 13, 13, 12, 12, 
          7, 12, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 13, 0, 12, 0, 0, 4, 9, 0, 0, 12, 
          9, 9, 13, 9, 4, 0, 9, 9, 9, 9, 9, 4, 4, 4, 5, 5, 4, 11, 6, 3, 11, 11, 
          5, 9, 11, 11, 6, 10, 6, 10, 1, 3, 11, 11, 3, 5, 3, 15, 6, 6, 16, 15, 
          6, 10, 10, 15, 6, 14, 6, 10, 10, 10, 10, 10, 10, 10, 1, 10, 10, 10, 
          10, 6, 16, 15, 11, 11, 15, 15, 11, 11, 15, 11, 1, 1, 1, 1, 1, 1, 1, 
          1, 1, 1, 16, 14, 16, 1, 15, 16, 14, 16, 16, 16, 14, 14, 16, 1, 14, 
          16, 14, 8, 11, 8, 16, 8, 16, 8, 5, 8, 14, 8, 16, 16, 16, 16, 16, 16, 
          16, 16, 14, 5, 4, 8, 5, 5, 5, 5, 4, 4, 8, 8, 8, 8, 8, 16, 8, 8, 4, 4, 
          4, 8, 5, 5, 5, 5, 8, 8, 8, 8, 8, 5, 12, 13, 5, 5, 0, 5, 5, 5, 0, 8, 
          8, 8, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 13, 13, 13, 13, 5, 5, 
          5, 13, 13, 13, 7, 7, 7, 7, 13, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 13, 13, 
          13, 13, 13, 13, 13, 7, 7, 7, 2, 2, 2, 7, 13, 13, 13, 7, 2, 7, 7, 13, 
          5, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 7, 13, 7, 2, 2, 2] 

# list of days in a year [0,1,2.... 364]
Days = list(range(0,len(Labels)))

In [2]:
# List of each cluster number
Unique_labels = list(set(Labels))
Unique_labels

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

## Looking at neighboors
* The idea behing this sorting algorythme is to look at the n nearest neighboors of each day
* The first assumption for n is the number of clusters so 17 in this case
* For each day we lookup the 17 preceeding days and the 17 days after and the cluster those days belong to
* For each day we set the cluster value equal to that of the most frequent cluster
* Note that the year is considered to be cyclical so for day 0 we check days 0 to 17 and days 365 - 17 to 365

In [3]:
def get_nearby(start, stop, Labels):
    if start >= 0 and stop <= len(Labels):
        return Labels[start:stop]
    else:
        if start <= 0:
            return Labels[len(Labels) + start:-1] + Labels[0:stop]
        else:
            return Labels[start:-1] + Labels[0:stop - len(Labels)]
        
def unique(l):
    return list(set([int(i) for i in l]))

In [4]:
count_near_day = {}
for d in Days:
    count_near_day[d] = {}
    Unique_labels = unique(Labels)
    l = len(Unique_labels)
    nearby = get_nearby(d - l, d + l, Labels)
    for p in unique(nearby):
        c = nearby.count(p)
        count_near_day[d][p] = c
        #print(f'Near day {d}, period {p} appears {c} times')

In [5]:
top_near_day = np.zeros(len(Days))
for d in Days:
    index = np.argmax(list(count_near_day[d].values()))
    top_near_day[d] = list(count_near_day[d].keys())[index]

## Reducing the distance
* at this point the distance is too great to represent at least one of each cluster in the year
* we can try for each distance smaller than 17 and stop as soon as each cluster in the year is present at least once
* in this cas it happens for a distance of 4 which corresponds to a range of 7 days.

In [6]:
distance = len(Unique_labels)
while len(unique(top_near_day)) != len(Unique_labels):
    count_near_day = {}
    for d in Days:
        count_near_day[d] = {}
        Unique_labels = unique(Labels)
        nearby = get_nearby(d - distance, d + distance, Labels)
        for p in unique(nearby):
            c = nearby.count(p)
            count_near_day[d][p] = c
            
    top_near_day = np.zeros(len(Days))
    for d in Days:
        index = np.argmax(list(count_near_day[d].values()))
        top_near_day[d] = list(count_near_day[d].keys())[index]
    
    distance -= 1

In [7]:
def reduce_reorder(order, period):
    period_occure = np.where(np.array(order) == period)[0]
    length = {}
    i = 0
    j = period_occure[0]
    for n, _ in enumerate(period_occure):
        try:
            if period_occure[n + 1] == period_occure[n] + 1:
                i += 1
            else:
                length[j] = i + 1
                i = 0
                j = period_occure[n+1]
        except:
            length[j] = i + 1
    key_max = max(length, key=length.get)
    for k in list(length.keys())[::-1]:
        l = length[k]
        if k == key_max:
            order[k:k+l] = [period]
        else:
            order[k:k+l] = []
    return order

## Option to set the distance
* the distance can be set manually
* however anything above 4 will result in a incomplete order
* if we leave a distance of 17 only the 13 largest clusters are ordered the remaining can be sorted by hand
* hand picking or leaving a distance of 4 does not change the final order

In [8]:
custon_distance = False
if custon_distance:
    distance = 4
    count_near_day = {}
    for d in Days:
        count_near_day[d] = {}
        Unique_labels = unique(Labels)
        nearby = get_nearby(d - distance, d + distance, Labels)
        for p in unique(nearby):
            c = nearby.count(p)
            count_near_day[d][p] = c

    top_near_day = np.zeros(len(Days))
    for d in Days:
        index = np.argmax(list(count_near_day[d].values()))
        top_near_day[d] = list(count_near_day[d].keys())[index]
    print(distance, top_near_day)

## Computing the order
* to get the order for each cluster we start with the most infrequent cluster
* we lookup where in the year it occure
* whenever it occurs we check how many days in a row it occurs
* whenever it occurs the longest those n days are replaced by a signle value of that cluster
* everywhere else that cluster is removed
* the same method is applied for the next most infrequent cluster unit all clusters have been checked

In [9]:
order = top_near_day

frequency = []
for i in unique(top_near_day):
    frequency.append(np.count_nonzero(top_near_day == i))
frequency = [unique(top_near_day), frequency]

order = list(top_near_day)
for i in unique(top_near_day):
    smallest = frequency[0].pop(np.argmin(frequency[1]))
    frequency[1].pop(np.argmin(frequency[1]))
    print('------------- Least frequent cluster: ', smallest, '-------------------------------------')
    print('---- Remaining elements of the least frequent cluster to be cut ', np.count_nonzero(np.array(order) == smallest))
    print(order)
    order = reduce_reorder(list(order), smallest)
    print('---- Remaining elements of the least frequent cluster after cut ', np.count_nonzero(np.array(order) == smallest))
    

------------- Least frequent cluster:  15 -------------------------------------
---- Remaining elements of the least frequent cluster to be cut  1
[2.0, 2.0, 2.0, 7.0, 7.0, 7.0, 2.0, 2.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 7.0, 7.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 7.0, 7.0, 7.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 0.0, 13.0, 13.0, 0.0, 13.0, 12.0, 0.0, 7.0, 13.0, 7.0, 7.0, 7.0, 7.0, 7.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 12.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 4.0, 4.0, 4.0, 4.0, 4.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 6.0, 6.0, 6.0, 6.0, 6.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0

## Result
* The resulting list contains one of each cluster for the entire year

In [10]:
# Ordered list of cluster numbers
order

[2, 12, 4, 0, 9, 11, 3, 6, 10, 15, 1, 16, 14, 8, 5, 7, 13]